In [ ]:
import os
import time
from shutil import move
import numpy as np
from subprocess import run
from subprocess import DEVNULL
from pathlib import Path as p
from multiprocessing import Pool
from shutil import move

In [2]:
work_dir = p('/media/Data/GoogleDriveNtust/dissertation_01/workFinal/workDir')
template = p('/media/Data/GoogleDriveNtust/dissertation_01/workFinal/templates/MNI152_T1_1mm_brain.nii')
template2mm = p('/media/Data/GoogleDriveNtust/dissertation_01/workFinal/templates/MNI152_T1_2mm_brain.nii')

In [3]:
derivatives = p(f'{work_dir}/test_dataset/derivatives')
binaries = os. getcwd()

os.chdir(derivatives)
os.chdir('../..')
results = os.getcwd()

outputResults = p(f'{results}/registration_results/processingTime_single')
try:
    os.mkdir(outputResults)
except OSError:
    pass

pipeline0 = p(f'{derivatives}/pipeline_0')
subject_pre = os.listdir(pipeline0)
pipeline1s = p(f'{derivatives}/pipeline_1_single')
try:
    os.mkdir(pipeline1s)
except OSError:
    pass

In [4]:
subject_pre.sort()
subject = subject_pre[0:10]
print(subject)

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10']


In [ ]:
print('First Loop out of Six')
os.write(1, "FIRST Loop out of SIX".encode())

subjectNumber = ['Subject Number']
processingTime = ['Ants Processing Time (seconds)']

ants = p(f'{pipeline1s}/ants')
try:
    os.mkdir(ants)
except OSError:
    pass

for i in range(len(subject)):
    
    antsSubject = p(f'{ants}/{subject[i]}')   
    try:
        os.mkdir(antsSubject)
    except OSError:
        pass
    
    movingImageName = os.listdir(f'{pipeline0}/{subject[i]}')
    movingImage = p(f'{pipeline0}/{subject[i]}/{movingImageName[0]}')

    os.chdir(antsSubject)
    
    start = time.time()
    
    rc = run(['antsRegistrationSyN.sh','-d', '3', '-f', f'{template}', '-m', f'{movingImage}', '-o', 'ants', '-n', '1'])
    
    end = time.time()
    processingTime.append(end - start)
    subjectNumber.append(subject[i])
          
    os.remove('ants0GenericAffine.mat')
    os.remove('ants1InverseWarp.nii.gz')
    os.remove('ants1Warp.nii.gz')
    os.remove('antsInverseWarped.nii.gz')
    
    print(end - start)    
    print('\n')
    print('@@@@@@@@@@@@@@')
fu = np.column_stack((subjectNumber, processingTime))

fubar = fu.transpose()

np.savetxt(p(f'{outputResults}/ANTS_time.csv'), fubar, fmt='%s', delimiter = ',')

In [ ]:
###ANTS QUICK LOOP

print('Second Loop out of Six')
os.write(1, "Second Loop out of SIX".encode())

subjectNumber = ['Subject Number']
processingTime = ['Ants Quick Processing Time (seconds)']

#ants Quick registration directory = ants
antsQuick = p(f'{pipeline1s}/antsQuick')
try:
    os.mkdir(antsQuick)
except OSError:
    pass

#loop for full ants reg

for i in range(len(subject)):   
    antsQuickSubject = p(f'{antsQuick}/{subject[i]}')    
    try:
        os.mkdir(antsQuickSubject)
    except OSError:
        pass
       
    movingImageName = os.listdir(f'{pipeline0}/{subject[i]}')
    movingImage = p(f'{pipeline0}/{subject[i]}/{movingImageName[0]}')

    os.chdir(antsQuickSubject)
    
    start = time.time()
    
    run(f'antsRegistrationSyNQuick.sh -d 3 -f {template} -m {movingImage} -o antsQuick -n 1', shell=True)
    
    end = time.time()
    processingTime.append(end - start)
    subjectNumber.append(subject[i])
    
    os.remove('antsQuick0GenericAffine.mat')
    os.remove('antsQuick1InverseWarp.nii.gz')
    os.remove('antsQuick1Warp.nii.gz')
    os.remove('antsQuickInverseWarped.nii.gz')
    
    print(end - start)
    print('\n')
    print('@@@@@@@@@@@@@@')
fu = np.column_stack((subjectNumber, processingTime))
fubar = fu.transpose()

np.savetxt(p(f'{outputResults}/ANTSQuick_time.csv'), fubar, fmt='%s', delimiter = ',')

In [5]:
#ELASTIX LOOP

print('Fourth Loop out of Six')
os.write(1, "Fourth Loop out of SIX".encode())

subjectNumber = ['Subject Number']
processingTime = ['elastix Processing Time (seconds)']

p1 = 'Parameters_Translation.txt'
p2 = 'Parameters_Rigid.txt'
p3 = 'Parameters_Affine.txt'
p4 = 'Parameters_BSpline.txt'

#elastix registration directory = elastix
elastix = p(f'{pipeline1s}/elastix')
try:
    os.mkdir(elastix)
except OSError:
    pass

#loop for full ants reg
for i in range(len(subject)):
    os.chdir(binaries)
    elastixSubject = p(f'{elastix}/{subject[i]}')

    try:
        os.mkdir(elastixSubject)
    except OSError:
        pass
    
    movingImageName = os.listdir(f'{pipeline0}/{subject[i]}')
    movingImage = p(f'{pipeline0}/{subject[i]}/{movingImageName[0]}')
    
    start = time.time()
    
    run(['elastix', '-f', f'{template}', '-m', f'{movingImage}', '-out', f'{elastixSubject}', '-p', f'{p1}','-p', f'{p2}','-p',f'{p3}','-p',f'{p4}', '-threads', '1'], stdout=DEVNULL)
    
    end = time.time()
    processingTime.append(end - start)
    subjectNumber.append(subject[i])
    
    os.chdir(elastixSubject)
    
    keepers = ['result.3.nii']

    for filename in os.listdir('.'):
        if filename not in keepers:
            print('Removing %s' % (filename,))
            os.remove(filename)
    
    print(end - start)
    print('\n')
    print('@@@@@@@@@@@@@@')
fu = np.column_stack((subjectNumber, processingTime))
fubar = fu.transpose()
np.savetxt(p(f'{outputResults}/Elastix_time.csv'), fubar, fmt='%s', delimiter = ',')

Fourth Loop out of Six
Removing elastix.log
Removing IterationInfo.0.R0.txt
Removing IterationInfo.0.R1.txt
Removing IterationInfo.0.R2.txt
Removing IterationInfo.0.R3.txt
Removing IterationInfo.1.R0.txt
Removing IterationInfo.1.R1.txt
Removing IterationInfo.1.R2.txt
Removing IterationInfo.1.R3.txt
Removing IterationInfo.2.R0.txt
Removing IterationInfo.2.R1.txt
Removing IterationInfo.2.R2.txt
Removing IterationInfo.2.R3.txt
Removing IterationInfo.3.R0.txt
Removing IterationInfo.3.R1.txt
Removing IterationInfo.3.R2.txt
Removing IterationInfo.3.R3.txt
Removing result.0.nii
Removing result.1.nii
Removing result.2.nii
Removing TransformParameters.0.txt
Removing TransformParameters.1.txt
Removing TransformParameters.2.txt
Removing TransformParameters.3.txt
102.10185313224792


@@@@@@@@@@@@@@
Removing elastix.log
Removing IterationInfo.0.R0.txt
Removing IterationInfo.0.R1.txt
Removing IterationInfo.0.R2.txt
Removing IterationInfo.0.R3.txt
Removing IterationInfo.1.R0.txt
Removing IterationInf

In [6]:
print('Fifth Loop out of Six')
os.write(1, "FIFTH Loop out of SIX".encode())

subjectNumber = ['Subject Number']
processingTime = ['SPMpy Processing Time (seconds)']

spmPy = p(f'{pipeline1s}/spmPy')
try:
    os.mkdir(spmPy)
except OSError:
    pass

for i in range(len(subject)):
    
    spmPySubject = p(f'{spmPy}/{subject[i]}')   
    try:
        os.mkdir(spmPySubject)
    except OSError:
        pass
    
    movingImageName = os.listdir(f'{pipeline0}/{subject[i]}')
    movingImage = p(f'{pipeline0}/{subject[i]}/{movingImageName[0]}')

    os.chdir(binaries)

    start = time.time()
    
    run(f'matlab -nodesktop -nosplash -singleCompThread -r "try spmPy(\'{movingImage}\', \'{template}\'); catch; end; quit"', shell=True)
    
    end = time.time()
    processingTime.append(end - start)
    subjectNumber.append(subject[i])

    os.chdir(spmPySubject)
    
    outputFile = p(f'{pipeline0}/{subject[i]}/spmPywmbet_{subject[i]}.nii')
    run(f'fslmaths {outputFile} -nan spmPy_{subject[i]}', shell=True)
    
    os.chdir(p(f'{pipeline0}/{subject[i]}'))
    keepers = [f'bet_{subject[i]}.nii', f'bet_{subject[i]}_to_MNI152_T1_1mm_brain.log']
    for filename in os.listdir('.'):
        if filename not in keepers:
            os.remove(filename)
    
    print(end - start)    
    print('\n')
    print('@@@@@@@@@@@@@@')
fu = np.column_stack((subjectNumber, processingTime))

fubar = fu.transpose()

np.savetxt(p(f'{outputResults}/SPMpy_time.csv'), fubar, fmt='%s', delimiter = ',')

Fifth Loop out of Six
147.72676587104797


@@@@@@@@@@@@@@
135.6505036354065


@@@@@@@@@@@@@@
133.2839548587799


@@@@@@@@@@@@@@
127.28683042526245


@@@@@@@@@@@@@@
135.25445890426636


@@@@@@@@@@@@@@
131.1555678844452


@@@@@@@@@@@@@@
139.1287567615509


@@@@@@@@@@@@@@
134.49164032936096


@@@@@@@@@@@@@@
121.12580895423889


@@@@@@@@@@@@@@
128.28200793266296


@@@@@@@@@@@@@@


In [7]:
def fsl_reg2mm(subjectNamesList):
    fslSubject = p(f'{fsl2mmThreaded}/{subjectNamesList}')
    try:
        os.mkdir(fslSubject)
    except OSError:
        pass
    
    movingImageName = os.listdir(f'{pipeline0}/{subjectNamesList}')  
    movingImage = p(f'{pipeline0}/{subjectNamesList}/{movingImageName[0]}')
    
    os.chdir(fslSubject)
    
    start = time.time()
    
    rc1 = run(f'flirt -ref {template2mm} -in {movingImage} -omat affine.mat', shell=True)
    
    matrixFile = f'{fslSubject}/affine.mat'
    
    rc2 = run(f'fnirt --in={movingImage} --aff={matrixFile} --cout=nonLinear --config=T1_2_MNI152_2mm', shell=True)
      
    nonLinear = f'{fslSubject}/nonLinear.nii.gz'

    rc3 = run(f'applywarp --ref={template} --in={movingImage} --warp={nonLinear} --out=warpedStructural', shell=True)

    
    end = time.time()
    processingTime = end - start
    
    try:
        os.remove('affine.mat')
    except OSError:
        pass
    try:
        os.remove('nonLinear.nii.gz')
    except OSError:
        pass
    
    return processingTime

In [8]:
subjectNumber = ['Subject Number']
processingTime = ['FSL2mm Processing Time (seconds)']

#fsl registration directory = ants
fsl2mmThreaded = p(f'{pipeline1s}/fsl2mmThreaded')
try:
    os.mkdir(fsl2mmThreaded)
except OSError:
    pass

#loop for full FSL Threaded reg
startThreaded = time.time() 

if __name__ == '__main__':
    pool = Pool(1)
    a = pool.map(fsl_reg2mm, subject)
    pool.close()
    pool.join()
    
endThreaded = time.time()


subjectNumber.extend(subject)
processingTime.extend(a)

fu = np.column_stack((subjectNumber, processingTime))
fubar = fu.transpose()
print('Overall 2mm Threaded Time')
print(endThreaded - startThreaded)
np.savetxt(p(f'{outputResults}/FSL2mmThreaded_time.csv'), fubar, fmt='%s', delimiter = ',')

Overall 2mm Threaded Time
4010.1259150505066
